In [1]:
import os

In [2]:
os.chdir("../")

In [3]:
!pip install dagshub mlflow

In [4]:
import dagshub
dagshub.init(repo_owner='Saketh1714', repo_name='End-End-ML-project', mlflow=True)

import mlflow
with mlflow.start_run():
  mlflow.log_param('parameter name', 'value')
  mlflow.log_metric('metric name', 1)

Accessing as Saketh1714

Initialized MLflow to track repo "Saketh1714/End-End-ML-project"

Repository Saketh1714/End-End-ML-project initialized!

🏃 View run mercurial-horse-411 at: https://dagshub.com/Saketh1714/End-End-ML-project.mlflow/#/experiments/0/runs/8adc881127d646cc833a70abd7b7ba04
🧪 View experiment at: https://dagshub.com/Saketh1714/End-End-ML-project.mlflow/#/experiments/0


In [5]:
import os
os.environ["MLFLOW_TRACKING_URI"] = "https://dagshub.com/Saketh1714/End-End-ML-project.mlflow"
os.environ["MLFLOW_TRACKING_USERNAME"] = "Saketh1714"
os.environ["MLFLOW_TRACKING_PASSWORD"] = "445a8da5dba78ff123c47605180edaa152574843"  # Replace with your DagsHub token

In [6]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class ModelEvaluationConfig:
    root_dir: Path
    test_data_path: Path
    model_path: Path
    all_params: dict
    metric_file_name: Path
    target_column: str
    mlflow_uri: str

In [7]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml, create_directories, save_json

In [8]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])

    
    def get_model_evaluation_config(self) -> ModelEvaluationConfig:
        config = self.config.model_evaluation
        params = self.params.ElasticNet
        schema =  self.schema.TARGET_COLUMN

        create_directories([config.root_dir])

        model_evaluation_config = ModelEvaluationConfig(
            root_dir=config.root_dir,
            test_data_path=config.test_data_path,
            model_path = config.model_path,
            all_params=params,
            metric_file_name = config.metric_file_name,
            target_column = schema.name,
            mlflow_uri="https://dagshub.com/Saketh1714/End-End-ML-project.mlflow",
           
        )

        return model_evaluation_config

In [9]:

import os
import pandas as pd
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score
from urllib.parse import urlparse
import mlflow
import mlflow.sklearn
import numpy as np
import joblib

In [10]:
class ModelEvaluation:
    def __init__(self, config: ModelEvaluationConfig):
        self.config = config

    
    def eval_metrics(self,actual, pred):
        rmse = np.sqrt(mean_squared_error(actual, pred))
        mae = mean_absolute_error(actual, pred)
        r2 = r2_score(actual, pred)
        return rmse, mae, r2
    


    def log_into_mlflow(self):

        test_data = pd.read_csv(self.config.test_data_path)
        model = joblib.load(self.config.model_path)

        test_x = test_data.drop([self.config.target_column], axis=1)
        test_y = test_data[[self.config.target_column]]


        mlflow.set_registry_uri(self.config.mlflow_uri)
        tracking_url_type_store = urlparse(mlflow.get_tracking_uri()).scheme


        with mlflow.start_run():

            predicted_qualities = model.predict(test_x)

            (rmse, mae, r2) = self.eval_metrics(test_y, predicted_qualities)
            
            # Saving metrics as local
            scores = {"rmse": rmse, "mae": mae, "r2": r2}
            save_json(path=Path(self.config.metric_file_name), data=scores)

            mlflow.log_params(self.config.all_params)

            mlflow.log_metric("rmse", rmse)
            mlflow.log_metric("r2", r2)
            mlflow.log_metric("mae", mae)


            # Log model without registering to avoid permission issues
            mlflow.sklearn.log_model(model, "model")


In [11]:
try:
    config = ConfigurationManager()
    model_evaluation_config = config.get_model_evaluation_config()
    model_evaluation_config = ModelEvaluation(config=model_evaluation_config)
    model_evaluation_config.log_into_mlflow()
except Exception as e:
    raise e

[2025-12-31 15:34:57,512: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-12-31 15:34:57,514: INFO: common: yaml file: params.yaml loaded successfully]
[2025-12-31 15:34:57,520: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-12-31 15:34:57,524: INFO: common: created directory at: artifacts]
[2025-12-31 15:34:57,526: INFO: common: created directory at: artifacts/model_evaluation]


C:\Users\madar\AppData\Roaming\Python\Python312\site-packages\sklearn\base.py:380: InconsistentVersionWarning: Trying to unpickle estimator ElasticNet from version 1.8.0 when using version 1.6.1. This might lead to breaking code or invalid results. Use at your own risk. For more info please refer to:
https://scikit-learn.org/stable/model_persistence.html#security-maintainability-limitations
  warnings.warn(


[2025-12-31 15:34:58,092: INFO: common: json file saved at: artifacts\model_evaluation\metrics.json]


2025/12/31 15:34:59 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


🏃 View run amusing-eel-340 at: https://dagshub.com/Saketh1714/End-End-ML-project.mlflow/#/experiments/0/runs/404d0b8db2eb4277a326c0689e2c574b
🧪 View experiment at: https://dagshub.com/Saketh1714/End-End-ML-project.mlflow/#/experiments/0
